In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, coalesce, isnan

# Configurações do Spark
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
# Recomenda-se o uso de secrets ou IAM Roles (o seu InstanceProfileCredentialsProvider está correto)
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

spark = SparkSession.builder.config(conf=conf).appName("UnificarUPA").getOrCreate()

# ======================
# ARQUIVOS E CAMINHOS
# ======================
s3_prefix = 's3a://bucket-raw-upa-connect-sofh/arquivos/'

arquivos_sensor = [
    '2025_10_13.csv',
    '2025_10_12.csv',
    '2025_10_11.csv',
    '2025_10_10.csv',
    '2025_10_09.csv'
]

# (Observação: 'arquivos_atendimento' não foi definido, mas foi mantido no código original)
arquivos_atendimento = [] # Assumindo lista vazia se não for usado, ou defina seus arquivos
colunas_finais = ['data_hora', 'valor', 'fk_sensor', 'fk_unid_medida', 'fk_paciente', 'fk_upa'] 


# ======================
# LEITURA DOS ARQUIVOS
# ======================

sensor_paths = [s3_prefix + f for f in arquivos_sensor]
atend_paths = [s3_prefix + f for f in arquivos_atendimento] # Pode dar erro se lista vazia e você tentar ler

sensor_df = spark.read.option('header', 'true').option('delimiter', ',').csv(sensor_paths)
# Leitura do atendimento_df omitida ou ajustada se não for usado para evitar erro
# atendimento_df = spark.read.option('header', 'true').option('delimiter', ',').csv(atend_paths) 
# Substituindo por um DF vazio se não for necessário, para evitar erro de leitura
atendimento_df = spark.createDataFrame([], schema=sensor_df.schema)


# Adicionar colunas que não existam em algum dos DataFrames
for col_name in colunas_finais:
    if col_name not in sensor_df.columns:
        sensor_df = sensor_df.withColumn(col_name, lit(None))
    if col_name not in atendimento_df.columns:
        atendimento_df = atendimento_df.withColumn(col_name, lit(None))

# Selecionar e ordenar as colunas
sensor_df = sensor_df.select(colunas_finais)


# ==========================================
# TRATAMENTO DE OUTLIERS E NULOS (SEÇÃO ADICIONADA)
# ==========================================

print(f"Total de linhas antes do tratamento: {sensor_df.count()}")

# 1. Cast de Tipos: Converte as colunas para o tipo correto para permitir comparações
sensor_df = sensor_df.withColumn("valor", col("valor").cast("double"))
sensor_df = sensor_df.withColumn("fk_sensor", col("fk_sensor").cast("int"))
sensor_df = sensor_df.withColumn("fk_unid_medida", col("fk_unid_medida").cast("int"))

# --- Condições de EXCLUSÃO ---
# Vamos construir as condições para identificar as linhas que DEVEM SER APAGADAS (o inverso do que queremos manter).

# 1. fk_sensor = 1 (Geral): apagar linhas em que valor veio NULO, maior que 300, e menor que 0
condicao_exclusao_1 = (
    (col("fk_sensor") == 1) &
    (
        col("valor").isNull() |           # NULO
        isnan(col("valor")) |            # NaN (Not a Number, se a conversão falhar)
        (col("valor") > 300) |           # Maior que 300
        (col("valor") < 0)               # Menor que 0
    )
)

# 2. fk_sensor = 1 e fk_unid_medida = 1: apagar linhas em que valor veio NULO, maior que 40 e menor que 0
condicao_exclusao_2 = (
    (col("fk_sensor") == 1) &
    (col("fk_unid_medida") == 1) &
    (
        col("valor").isNull() |
        isnan(col("valor")) |
        (col("valor") > 40) |            # Maior que 40
        (col("valor") < 0)
    )
)

# 3. fk_sensor = 1 e fk_unid_medida = 2: apagar linhas em que valor veio NULO, maior que 100 e menor que 0
# (Corrigindo "fk_unid_media" para "fk_unid_medida")
condicao_exclusao_3 = (
    (col("fk_sensor") == 1) &
    (col("fk_unid_medida") == 2) &
    (
        col("valor").isNull() |
        isnan(col("valor")) |
        (col("valor") > 100) |           # Maior que 100
        (col("valor") < 0)
    )
)

# Combinação das condições: Uma linha deve ser APAGADA se atender a QUALQUER uma das condições
condicao_exclusao_total = condicao_exclusao_1 | condicao_exclusao_2 | condicao_exclusao_3

# Aplicar o filtro: Manter apenas as linhas onde a condição de exclusão NÃO é verdadeira.
sensor_df = sensor_df.filter(~condicao_exclusao_total)

# ======================
# RESULTADOS
# ======================
sensor_df.show(10)
print(f"Total de linhas após o tratamento: {sensor_df.count()}")

# Parar a sessão Spark ao final do notebook
# spark.stop()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f6626744-9274-4263-81f3-defcd2e82f92;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 551ms :: artifacts dl 20ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

Total de linhas antes do tratamento: 293755
+-------------------+-----+---------+--------------+-----------+------+
|          data_hora|valor|fk_sensor|fk_unid_medida|fk_paciente|fk_upa|
+-------------------+-----+---------+--------------+-----------+------+
|2025-10-13T00:05:00| 17.0|        1|          null|       null|     1|
|2025-10-13T00:10:00| 15.0|        1|          null|       null|     1|
|2025-10-13T00:15:00| 12.0|        1|          null|       null|     1|
|2025-10-13T00:20:00| 12.0|        1|          null|       null|     1|
|2025-10-13T00:25:00| 10.0|        1|          null|       null|     1|
|2025-10-13T00:45:00| 12.0|        1|          null|       null|     1|
|2025-10-13T00:50:00| 13.0|        1|          null|       null|     1|
|2025-10-13T00:55:00| 10.0|        1|          null|       null|     1|
|2025-10-13T01:10:00|  3.0|        1|          null|       null|     1|
|2025-10-13T01:15:00|  9.0|        1|          null|       null|     1|
+-------------------

Total de linhas após o tratamento: 242846


In [2]:
# tabela_unificada.coalesce(1) \
#     .write.mode('overwrite') \
#     .option('header', 'true') \
#     .csv('s3a://bucket-trusted-upa-connect-sofh/tabela_sensores_tratada.csv')